In [2]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.2'
spark_version = 'spark-3.2.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://security.ubuntu.com/ubuntu bionic-security/main amd64 Packages [3,068 kB]
Hit:13 http://ppa.la

In [3]:
# Download Postgres Driver
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-11-18 06:08:26--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  6.18MB/s    in 0.2s    

2022-11-18 06:08:27 (6.18 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [4]:
# Import SparkSession
from pyspark.sql import SparkSession
# Start Spark Session with Added Postgres Driver
spark = SparkSession.builder.appName("LoadDB").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [5]:
# Read in json data with user id from S3 Bucket
from pyspark import SparkFiles
url ="https://gun-sentiment-analysis.s3.us-west-2.amazonaws.com/twitter_training_data/training_tweets_raw.json"
spark.sparkContext.addFile(url)
raw_tweet_df = spark.read.json(SparkFiles.get("training_tweets_raw.json"))

In [6]:
raw_tweet_df.show()

+------------+-----------+--------------------+------------------+------------------+--------------------+--------+--------------------+--------------------+--------------------+--------------+---------+------------+----+-------------------+-------------------+-----------------------+---------------------+-------------------------+-------------------+-----------------------+---------------+----+--------------+-----+------------------+-----------+-----------+-------------+---------+------+--------------------+--------------------+---------+--------------------+
|contributors|coordinates|          created_at|display_text_range|     edit_controls|        edit_history|editable|            entities|   extended_entities|      extended_tweet|favorite_count|favorited|filter_level| geo|                 id|             id_str|in_reply_to_screen_name|in_reply_to_status_id|in_reply_to_status_id_str|in_reply_to_user_id|in_reply_to_user_id_str|is_quote_status|lang|matching_rules|place|possibly_sensi

In [7]:
tweet_id_user_df = raw_tweet_df.selectExpr(["id_str as tweet_id", "user"])
tweet_id_user_df.show()

+-------------------+--------------------+
|           tweet_id|                user|
+-------------------+--------------------+
|1587957708166426625|{false, Sun Jul 0...|
|1587957661085171712|{false, Fri Jan 1...|
|1587957440494141441|{false, Thu Jan 2...|
|1587957427823349760|{false, Sun Sep 1...|
|1587957397422841861|{false, Mon Sep 0...|
|1587957395539611648|{false, Fri May 1...|
|1587957387503501312|{false, Sat Jul 1...|
|1587957369543204865|{false, Tue Oct 2...|
|1587957359636189185|{false, Fri Aug 2...|
|1587957220502822912|{false, Wed Oct 0...|
|1587957194443685889|{false, Sat Oct 0...|
|1587957099048443904|{false, Sat Feb 2...|
|1587957099048275969|{false, Tue Nov 1...|
|1587957050440491008|{false, Mon Sep 1...|
|1587957045784973312|{false, Fri Feb 0...|
|1587957022980546560|{false, Sun Apr 1...|
|1587957018551357443|{false, Thu Feb 1...|
|1587956871792467969|{false, Sat Aug 0...|
|1587956833746034690|{false, Sat Jul 3...|
|1587956778846679041|{false, Mon Apr 0...|
+----------

In [8]:
tweet_id_user_df.printSchema()

root
 |-- tweet_id: string (nullable = true)
 |-- user: struct (nullable = true)
 |    |-- contributors_enabled: boolean (nullable = true)
 |    |-- created_at: string (nullable = true)
 |    |-- default_profile: boolean (nullable = true)
 |    |-- default_profile_image: boolean (nullable = true)
 |    |-- derived: struct (nullable = true)
 |    |    |-- locations: array (nullable = true)
 |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |-- country: string (nullable = true)
 |    |    |    |    |-- country_code: string (nullable = true)
 |    |    |    |    |-- full_name: string (nullable = true)
 |    |    |    |    |-- geo: struct (nullable = true)
 |    |    |    |    |    |-- coordinates: array (nullable = true)
 |    |    |    |    |    |    |-- element: double (containsNull = true)
 |    |    |    |    |    |-- type: string (nullable = true)
 |    |    |    |    |-- locality: string (nullable = true)
 |    |    |    |    |-- region: string (nullable 

In [9]:
user_id_df = tweet_id_user_df.select(["tweet_id", "user.id_str"])
user_id_df.show()

+-------------------+-------------------+
|           tweet_id|             id_str|
+-------------------+-------------------+
|1587957708166426625|         2677654265|
|1587957661085171712|           12096932|
|1587957440494141441|           19739355|
|1587957427823349760|           16283923|
|1587957397422841861|          187451789|
|1587957395539611648| 862874466333986816|
|1587957387503501312|         2660741214|
|1587957369543204865|         4003610356|
|1587957359636189185|1563305202412728320|
|1587957220502822912|          385373318|
|1587957194443685889|1578731482234978306|
|1587957099048443904| 967228018379907075|
|1587957099048275969|1062380306802462721|
|1587957050440491008|            8926282|
|1587957045784973312|1225898684765831168|
|1587957022980546560|1513157814142877707|
|1587957018551357443|         2342016626|
|1587956871792467969|1025715968461271046|
|1587956833746034690| 759495379301769216|
|1587956778846679041|         2431547731|
+-------------------+-------------

In [10]:
user_id_df = user_id_df.selectExpr(["tweet_id", "id_str as user_id"])
user_id_df.show()

+-------------------+-------------------+
|           tweet_id|            user_id|
+-------------------+-------------------+
|1587957708166426625|         2677654265|
|1587957661085171712|           12096932|
|1587957440494141441|           19739355|
|1587957427823349760|           16283923|
|1587957397422841861|          187451789|
|1587957395539611648| 862874466333986816|
|1587957387503501312|         2660741214|
|1587957369543204865|         4003610356|
|1587957359636189185|1563305202412728320|
|1587957220502822912|          385373318|
|1587957194443685889|1578731482234978306|
|1587957099048443904| 967228018379907075|
|1587957099048275969|1062380306802462721|
|1587957050440491008|            8926282|
|1587957045784973312|1225898684765831168|
|1587957022980546560|1513157814142877707|
|1587957018551357443|         2342016626|
|1587956871792467969|1025715968461271046|
|1587956833746034690| 759495379301769216|
|1587956778846679041|         2431547731|
+-------------------+-------------

In [12]:
from pyspark.sql import functions as f

unique_user_id_df = user_id_df.dropDuplicates(['user_id'])

print(unique_user_id_df.select(f.count("user_id")).show())

+--------------+
|count(user_id)|
+--------------+
|          6711|
+--------------+

None


In [13]:
print(f"unique_user_id_df dataframe row count: {unique_user_id_df.count()}")

unique_user_id_df dataframe row count: 6711


In [14]:
# count nulls
null_count_df = user_id_df.select([f.count(f.when(f.col(c).contains('None') | \
                            f.col(c).contains('NULL') | \
                            (f.col(c) == '' ) | \
                            f.col(c).isNull() | \
                            f.isnan(c), c 
                           )).alias(c)
                    for c in user_id_df.columns])

null_count_df.show()

+--------+-------+
|tweet_id|user_id|
+--------+-------+
|       0|      0|
+--------+-------+



In [119]:
# Read in training tweets data from S3 Bucket
url = "https://gun-sentiment-analysis.s3.us-west-2.amazonaws.com/twitter_training_data/training_tweets.csv"
spark.sparkContext.addFile(url)
tweet_df = spark.read.csv(SparkFiles.get("training_tweets.csv"), sep=",", multiLine=True, header=True, inferSchema=True)

In [120]:
tweet_df.show(truncate=False)

+-------------------+------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------+--------------------------------------------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------+---------------------+----------+----------+---------------+------------+-------+-------------+-----------------+-----------------+----------------+------------------+--------------------------------------------+--------------+--------------+-----------+-----------+-----------+--------------+------------------------------------------------------+
|tweet_id           |date                          |ful

In [121]:
# count nulls
tweet_null_count_df = tweet_df.select([f.count(f.when(f.col(c).contains('None') | \
                            f.col(c).contains('NULL') | \
                            (f.col(c) == '' ) | \
                            f.col(c).isNull() | \
                            f.isnan(c), c 
                           )).alias(c)
                    for c in tweet_df.columns])


tweet_null_count_df.select("tweet_id").show()

+--------+
|tweet_id|
+--------+
|       0|
+--------+



In [122]:
tweet_df.count()

9231

In [123]:
tweet_df.select(f.count("tweet_id")).show()

+---------------+
|count(tweet_id)|
+---------------+
|           9231|
+---------------+



In [149]:
tweet_tail = tweet_df.tail(50)
rdd = spark.sparkContext.parallelize(tweet_tail)
tweet_tail_df = spark.createDataFrame(rdd)
tweet_tail_df.show(truncate=False)

+-------------------+------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------+--------------------------------------------------------------------------------------------------+--------------------------+-----------+----------+---------------+------------+-------------+-------------+-----------------+-------------+--------------+-------------------+-----------------------------------------+-----------------------------+--------------+-----------+-----------+-----------+--------------+-----------------------------------------

In [126]:
tweets_user_id_df = tweet_df.join(user_id_df, on="tweet_id", how="inner")

tweets_user_id_df.show(truncate=False)

+-------------------+------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------+--------------------------------------------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------+---------------------+----------+----------+---------------+------------+-------+-------------+-----------------+-----------------+----------------+------------------+--------------------------------------------+--------------+--------------+-----------+-----------+-----------+--------------+------------------------------------------------------+-------------------+
|tweet_id           |date          

In [127]:
tweets_user_id_df.count()

8801

In [131]:
tweets_user_geo_df = tweet_df.join(unique_user_id_df, on="tweet_id", how="inner")

tweets_user_geo_df.show(truncate=False)

+-------------------+------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------+--------------------------------------------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------+---------------------+----------+----------+---------------+------------+-------+-------------+-----------------+-----------------+----------------+------------------+--------------------------------------------+--------------+--------------+-----------+-----------+-----------+--------------+------------------------------------------------------+-------------------+
|tweet_id           |date          

In [132]:
tweets_user_geo_df.count()

6711

In [133]:
# count nulls
tweets_user_id_null_count_df = tweets_user_id_df.select([f.count(f.when(f.col(c).contains('None') | \
                            f.col(c).contains('NULL') | \
                            (f.col(c) == '' ) | \
                            f.col(c).isNull() | \
                            f.isnan(c), c 
                           )).alias(c)
                    for c in tweets_user_id_df.columns])


tweets_user_id_null_count_df.show()

+--------+----+---------+----------+--------+--------+-------------+----------+----------+---------------+------------+-------+------------+-----------------+-------------+-----------+---------------+--------------+--------------+--------------+-----------+-----------+-----------+--------------+---------+-------+
|tweet_id|date|full_text|tweet_type|hashtags|mentions|user_location|place_type|place_name|place_full_name|country_code|country|prof_country|prof_country_code|prof_locality|prof_region|prof_sub_region|prof_full_name|prof_coord_lat|prof_coord_lng|reply_count|quote_count|likes_count|retweet_counts|hyperlink|user_id|
+--------+----+---------+----------+--------+--------+-------------+----------+----------+---------------+------------+-------+------------+-----------------+-------------+-----------+---------------+--------------+--------------+--------------+-----------+-----------+-----------+--------------+---------+-------+
|       0|   0|       14|       150|    7347|    2790| 

In [134]:
# count nulls
tweets_user_geo_null_count_df = tweets_user_geo_df.select([f.count(f.when(f.col(c).contains('None') | \
                            f.col(c).contains('NULL') | \
                            (f.col(c) == '' ) | \
                            f.col(c).isNull() | \
                            f.isnan(c), c 
                           )).alias(c)
                    for c in tweets_user_geo_df.columns])


tweets_user_geo_null_count_df.show()

+--------+----+---------+----------+--------+--------+-------------+----------+----------+---------------+------------+-------+------------+-----------------+-------------+-----------+---------------+--------------+--------------+--------------+-----------+-----------+-----------+--------------+---------+-------+
|tweet_id|date|full_text|tweet_type|hashtags|mentions|user_location|place_type|place_name|place_full_name|country_code|country|prof_country|prof_country_code|prof_locality|prof_region|prof_sub_region|prof_full_name|prof_coord_lat|prof_coord_lng|reply_count|quote_count|likes_count|retweet_counts|hyperlink|user_id|
+--------+----+---------+----------+--------+--------+-------------+----------+----------+---------------+------------+-------+------------+-----------------+-------------+-----------+---------------+--------------+--------------+--------------+-----------+-----------+-----------+--------------+---------+-------+
|       0|   0|       10|       105|    5680|    2169| 

In [135]:
# data types
tweets_user_id_df.dtypes

[('tweet_id', 'string'),
 ('date', 'string'),
 ('full_text', 'string'),
 ('tweet_type', 'string'),
 ('hashtags', 'string'),
 ('mentions', 'string'),
 ('user_location', 'string'),
 ('place_type', 'string'),
 ('place_name', 'string'),
 ('place_full_name', 'string'),
 ('country_code', 'string'),
 ('country', 'string'),
 ('prof_country', 'string'),
 ('prof_country_code', 'string'),
 ('prof_locality', 'string'),
 ('prof_region', 'string'),
 ('prof_sub_region', 'string'),
 ('prof_full_name', 'string'),
 ('prof_coord_lat', 'string'),
 ('prof_coord_lng', 'string'),
 ('reply_count', 'string'),
 ('quote_count', 'string'),
 ('likes_count', 'string'),
 ('retweet_counts', 'string'),
 ('hyperlink', 'string'),
 ('user_id', 'string')]

In [136]:
tweets_user_geo_df.dtypes

[('tweet_id', 'string'),
 ('date', 'string'),
 ('full_text', 'string'),
 ('tweet_type', 'string'),
 ('hashtags', 'string'),
 ('mentions', 'string'),
 ('user_location', 'string'),
 ('place_type', 'string'),
 ('place_name', 'string'),
 ('place_full_name', 'string'),
 ('country_code', 'string'),
 ('country', 'string'),
 ('prof_country', 'string'),
 ('prof_country_code', 'string'),
 ('prof_locality', 'string'),
 ('prof_region', 'string'),
 ('prof_sub_region', 'string'),
 ('prof_full_name', 'string'),
 ('prof_coord_lat', 'string'),
 ('prof_coord_lng', 'string'),
 ('reply_count', 'string'),
 ('quote_count', 'string'),
 ('likes_count', 'string'),
 ('retweet_counts', 'string'),
 ('hyperlink', 'string'),
 ('user_id', 'string')]

In [137]:
# tweets table df

tweets_df = tweets_user_id_df.selectExpr("tweet_id", "user_id", "date", "full_text", "tweet_type", "hashtags", "mentions", "cast(reply_count as int) reply_count", \
                                "cast(quote_count as int) quote_count", "cast(likes_count as int) likes_count", "cast(retweet_counts as int) retweet_counts", \
                                "hyperlink")

# tweets_df = tweets_df.withColumn("sentiment", f.lit(None))

tweets_df.dtypes

[('tweet_id', 'string'),
 ('user_id', 'string'),
 ('date', 'string'),
 ('full_text', 'string'),
 ('tweet_type', 'string'),
 ('hashtags', 'string'),
 ('mentions', 'string'),
 ('reply_count', 'int'),
 ('quote_count', 'int'),
 ('likes_count', 'int'),
 ('retweet_counts', 'int'),
 ('hyperlink', 'string')]

In [139]:
tweets_df.count()

8801

In [141]:
tweets_df.select(f.countDistinct("tweet_id")).show()

+------------------------+
|count(DISTINCT tweet_id)|
+------------------------+
|                    8801|
+------------------------+



In [157]:
tweets_df.show(truncate=False)

+-------------------+-------------------+------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------+--------------------------------------------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------+-----------+-----------+-----------+--------------+------------------------------------------------------+
|tweet_id           |user_id            |date                          |full_text                                                                                                                                                                                                    

In [158]:
# users_geo table df

users_geo_df = tweets_user_geo_df.selectExpr("user_id", "prof_country_code as country_code", "prof_full_name as location_name", "cast(prof_coord_lat as float) latitude", \
                                            "cast(prof_coord_lng as float) longitude")

users_geo_df.dtypes

[('user_id', 'string'),
 ('country_code', 'string'),
 ('location_name', 'string'),
 ('latitude', 'float'),
 ('longitude', 'float')]

In [160]:
users_geo_df.show(truncate=False)

+-------------------+------------+--------------------------------------------+----------+---------+
|user_id            |country_code|location_name                               |latitude  |longitude|
+-------------------+------------+--------------------------------------------+----------+---------+
|2677654265         |US          |Indianapolis, Indiana, United States        |-86.15804 |39.76838 |
|12096932           |US          |New Jersey, United States                   |-74.49987 |40.16706 |
|19739355           |US          |Kane, Illinois, United States               |-90.35373 |39.19005 |
|16283923           |US          |Washington, D.C., United States             |-77.00025 |38.91706 |
|187451789          |US          |Douglasville, Georgia, United States        |-84.74771 |33.7515  |
|862874466333986816 |US          |Florida, United States                      |-82.5001  |28.75054 |
|2660741214         |US          |Johnston, Iowa, United States               |-93.69772 |4

In [22]:
# Store environmental variable
from getpass import getpass
password = getpass('Enter database password')
# Configure settings for RDS
mode = "overwrite"
jdbc_url="jdbc:postgresql://gun-sentiment-db.ckbz1nmb8sin.us-west-2.rds.amazonaws.com:5432/gun_sentiment_tweets_db"
config = {"user":"postgres",
          "password": password,
          "driver":"org.postgresql.Driver"}

Enter database password··········


In [110]:
# Write tweet_df to tweets table
tweets_df.write.jdbc(url=jdbc_url, table='tweets', mode=mode, properties=config)

In [111]:
# Write users_geo_df to tweets table
users_geo_df.write.jdbc(url=jdbc_url, table='users_geo', mode=mode, properties=config)